# Process Data Table

This notebook takes patient data and processes it into the format suggested by the flexsurv R package for fitting a multi-state survival model.

Start by Pulling in relevant libraries.


In [ ]:
#### load packages
%matplotlib inline
import numpy as np
import scipy as sp
import scipy.stats as st
import scipy.optimize as op
from scipy import integrate
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, date 
from datetime import timedelta
from tqdm.notebook import tqdm

In [ ]:
#### Define dates
#### When extracting data, don't forget to remove patients admission in the last couple of days. 
#### This is important, since many patients tested on these days may still be waiting for their
#### test results, so the data won't accurately reflect the true number of COVID admissions. 

file_date = 'simulated_30May' # update to extract date
dtpull = datetime.fromisoformat('2020-05-27 23:59:00') # update to extract date, best to use 2 days prior to current date
dtcensor = datetime.fromisoformat('2020-05-30 23:59:00') # update to last day on which data is updated

#### Load data
df = pd.read_csv('Ward_COVID_'+file_date+'.csv', header = None, names=["NHSNumber", "HospitalAdmissionTime", "StartTimeCriticalCare", "DischargeTimeCriticalCare", "HospitalDischargeTime",  "DateOfDeath", "DateOfBirth"])


In [ ]:
#### Convert string dates to datetimes

ld=len(df)
dob_vec=df['DateOfBirth'].values

for i in range(0,ld):
    dob = df.DateOfBirth.values[i]
    adt = df.HospitalAdmissionTime.values[i]
    at = df.StartTimeCriticalCare.values[i]
    dt = df.DischargeTimeCriticalCare.values[i]
    mt = df.DateOfDeath.values[i]
    hdt = df.HospitalDischargeTime.values[i]
    if (not pd.isna(at)):
        df.StartTimeCriticalCare.iloc[i] =  datetime.fromisoformat(at)
    if (not pd.isna(dt)):
        df.DischargeTimeCriticalCare.iloc[i] = datetime.fromisoformat(dt)
    if (not pd.isna(mt)):
        df.DateOfDeath.iloc[i] = datetime.fromisoformat(mt)
    if (not pd.isna(hdt)):
        df.HospitalDischargeTime.iloc[i] = datetime.fromisoformat(hdt) 
    df.HospitalAdmissionTime.iloc[i] = datetime.fromisoformat(adt) 
    df.DateOfBirth.iloc[i] = datetime.fromisoformat(dob)


In [ ]:
#### For each NHS Number, extract the patient's first admission and final discharge
####(can be omitted if interested in tracking multiple admissions)
#### It is probably best to keep this in, as in the (MFT) data most individuals are quickly readmitted...
#### ...suggesting discharge decision was incorrect rather than a genuine state transition
   
ids = df['NHSNumber'].unique()
    
for i in tqdm(ids): 
    NHS_df = df.loc[df['NHSNumber'] == i] # find 
    first_admission = NHS_df['HospitalAdmissionTime'].min()
    

    # if ICU admission exists, remove all duplicates without ICU times
    if NHS_df['StartTimeCriticalCare'].notnull().values.any(): 
        removal = NHS_df.index[NHS_df['StartTimeCriticalCare'].isnull()].tolist()
        df = df.drop(removal).reset_index(drop=True)
    NHS_df = df.loc[df['NHSNumber'] == i] # find 
    first_icu_admission = NHS_df['StartTimeCriticalCare'].min()
 
    # find last discharge time
    if NHS_df['HospitalDischargeTime'].isnull().values.any(): 
        last_discharge = float('nan')
    else:
        last_discharge = NHS_df['HospitalDischargeTime'].max() # otherwie set as max discharge time
    removal1 = NHS_df.index[NHS_df['HospitalAdmissionTime'] > first_admission].tolist()
    
    # find last ICU discharge time
    if NHS_df['DischargeTimeCriticalCare'].isnull().values.any(): 
        last_icu_discharge = float('nan') # last ICU discharge NaN
    else:
        last_icu_discharge = NHS_df['DischargeTimeCriticalCare'].max() # last ICU discharge exists
    removal2 = NHS_df.index[NHS_df['StartTimeCriticalCare'] > first_icu_admission].tolist()
    
    if (not pd.isna(NHS_df['DateOfDeath'].values.any())):
        removal3 = NHS_df.index[NHS_df['DateOfDeath'].isnull()].tolist()
    else:
        removal3 = []
    
    # remove extra hospital admissions and apend last discharge to first admission
    df = df.drop(sorted(np.unique(removal1+removal2+removal3))).reset_index(drop=True)
    df.loc[df['NHSNumber'] == i,'HospitalDischargeTime']=last_discharge
    # apend last ICU discharge to first ICU admission
    df.loc[df['NHSNumber'] == i,'DischargeTimeCriticalCare']=last_icu_discharge
    NHS_df = df.loc[df['NHSNumber'] == i] # find 

    # check death date, if this is within 5 days of discharge, treat this as a death in hospital and amend discharge time
    if (NHS_df['DateOfDeath'].notnull().values.any()):
        DeathDate = NHS_df['DateOfDeath'].min()
        if (DeathDate > last_discharge) & (DeathDate < last_discharge+timedelta(5)):
            df.loc[df['NHSNumber'] == i,'HospitalDischargeTime']=DeathDate
            
df = df.drop_duplicates()


In [ ]:
#### Matrix of allowed transitions
Q = [             
    [0,1,0,1,1],
    [0,0,1,0,1],
    [0,0,0,1,0],
    [0,0,0,0,0],
    [0,0,0,0,0]
] 

In [ ]:
#### Create a data frame - each row corresponds to a transition made by an individual along with time of transition since admission
# and covariate values.

new_df = pd.DataFrame()
ids = df['NHSNumber'].unique()

for l, i in tqdm(enumerate(ids)):
    
    # Extracting relevant times for an individual
    NHS_df = df.loc[df['NHSNumber'] == i]
    if(len(NHS_df)>1):
        print(i)
        NHS_df = NHS_df.dropna(axis = 0, subset = ["StartTimeCriticalCare"])
    HospitalStay = NHS_df.HospitalAdmissionTime.values[0] 
    adt = NHS_df.HospitalAdmissionTime.values[0] 
    ICU_admit = NHS_df.StartTimeCriticalCare.values
    ICU_discharge = NHS_df.DischargeTimeCriticalCare.values[-1]
    NHS_discharge = NHS_df.HospitalDischargeTime.values[-1]
    dob = NHS_df.DateOfBirth.values[0]  
    NHS_death = NHS_df.DateOfDeath.values[-1]
    States = np.ones(1) 

    
    
     
    if (ICU_admit[0] == adt):  # addition to amend the zero durations, as the model doesn't deal with zeros  
        HospitalStay = ICU_admit[0] - timedelta(0.1)
        adt = ICU_admit[0] - timedelta(0.1)
    
     
    # Logical indicator for death in hospital ward and not an ICU    
    if (pd.isna(ICU_admit[0])): 
        if (not pd.isna(NHS_discharge)):
            Hosp_death = ((not pd.isna(NHS_death)) and NHS_death<=NHS_discharge)
        else:
            Hosp_death = (not pd.isna(NHS_death))
        
    elif (not pd.isna(ICU_discharge)):
        if (not pd.isna(NHS_discharge)):
            Hosp_death = ((not pd.isna(NHS_death)) and NHS_death<=NHS_discharge 
                      and ICU_discharge<NHS_discharge) or (ICU_discharge<NHS_discharge)
        else:
            Hosp_death = (not pd.isna(NHS_death))
    else:
        Hosp_death = False
    # Logical indicator for death in an ICU
    if (not pd.isna(ICU_discharge) and not pd.isna(NHS_death)):
        ICU_death = (not pd.isna(ICU_admit[0]) and not Hosp_death and not (pd.isna(NHS_death) and NHS_death <=ICU_discharge))
    else:
        ICU_death = False
    
    if (Hosp_death == False and ICU_death == False and ICU_discharge == NHS_discharge):
        NHS_discharge = ICU_discharge + timedelta(0.1)
            
        
    
    # Create new row for each transition, with corresponding state
    if (not pd.isna(ICU_admit[0])): # Do you have an ICU stay?
        
            CriticalCare = NHS_df[['StartTimeCriticalCare','DischargeTimeCriticalCare']].values
            CriticalCare = CriticalCare[~pd.isna(CriticalCare)]
            
            n_rows=np.size(CriticalCare) # Each element should have its own row
            CriticalCare=np.resize(CriticalCare, (n_rows, 1))
            CC_states = np.zeros(len(CriticalCare))
            
            
            
            for k in range(0, n_rows): # States alternate between 2 and 3 if multiple ICU visits exist
                if (k%2 == 0):
                    CC_states[k] = 2               
                else:
                    CC_states[k] = 3

                    
                    
            # Has a person died in the ICU?        
            if((not pd.isna(NHS_death))):
                if(pd.isna(ICU_discharge)):
                    CC_states[-1] = 5
                elif(NHS_death<=ICU_discharge):
                    CC_states[-1] = 5
            
            # Deal with case where individuals are admitted straight to ICU    
            if (ICU_admit[0] == adt): 
                HospitalStay = CriticalCare # - timedelta(0.1)
                States = CC_states.T
            else:
                HospitalStay = np.append(HospitalStay, CriticalCare) # Append hospital admission and ICU times
                States = np.append(States, CC_states) 
    
    # End state, 5 corresponds to death, 4 to discharge
    if ((Hosp_death and (not pd.isna(NHS_death) ) ) ): 
        States = np.append(States, 5)
    elif (not ((not pd.isna(NHS_death)) and (pd.isna(ICU_discharge) or NHS_death<=ICU_discharge or pd.isna(ICU_admit[0]))) 
          and (not pd.isna(NHS_discharge)) or (NHS_death>NHS_discharge)):
        States = np.append(States, 4)

    
    # Check this is a discharge (not transfer/death)
    if((not pd.isna(NHS_discharge)) and (NHS_discharge !=ICU_discharge) ): 
        HospitalStay = np.append(HospitalStay, NHS_discharge)
    
    # Reshape into desired format for data frame
    if (isinstance(HospitalStay, date)):
        HospitalStay = np.array([HospitalStay])
    else:
        HospitalStay = np.resize(HospitalStay, (len(HospitalStay),1))
    
     
    # Creates a covariate that is set to 1 after a patient is discharged from ICU
    num_events = len(HospitalStay)
    ICU_covariate = np.ones(num_events) 
    if (ICU_admit[0]==adt):
        ICU_covariate[0] = 0
    else:
        ICU_covariate[0:np.minimum(num_events,2)] = 0
        
    # Categorising individuals into age groups
    age = np.floor((adt - dob).days/365)
    if (age < 26 and age>0):
        age = 0
    elif(age <51 and age>25):
        age = 1
    elif(age<76 and age>50):
        age = 2
    elif(age>75):
        age = 3
  
    
    ages = np.repeat(age, num_events)
    adt = np.repeat(adt, num_events)
    
    timepull = (dtcensor - HospitalStay.flatten()[-1]).total_seconds()/(60*60*24)    
    # Create the full data frame for an individual
    try: # Catches errors for individuals discharged directly from ICU (assumed to be transfers) #### look at removing this
        for k in range(0, num_events):
            HospitalStay[k] = ((HospitalStay[k] - adt)[0].total_seconds())/(60*60*24)
        if (num_events==1):
            HospitalStay = np.append(HospitalStay, [ages[0], ICU_covariate[0], States[0]])
            HospitalStay = pd.DataFrame(HospitalStay).T
        else:
            HospitalStay = np.concatenate([HospitalStay,np.atleast_2d(ages).T, np.atleast_2d(ICU_covariate).T, 
                                           np.atleast_2d(States).T], axis = 1)
            HospitalStay = pd.DataFrame(HospitalStay)

        # Creates number of censoring rows based on possible transitions from each state
        k=0
        ndups = []
        Qbase = Q.copy()
        for c in range(0,len(Q)):
            ndups.append(0)
            for j in range(0,len(Q[c])):
                if Q[c][j] == 1:
                    k += 1
                    Qbase[c][j] = k
                    ndups[c] +=1
                    
        duplicated_df_vals = []  

        for b in range(0,len(HospitalStay)):  
            row = HospitalStay.iloc[b,:].values
            ii = int(HospitalStay.iloc[b,3])
            nreps = ndups[ii-1]
            if (b<(len(HospitalStay)-1)): # Not an end
                jj = int(HospitalStay.iloc[b+1,3])
                for qi, qval in enumerate(Qbase[ii-1]):
                    if (qval>0):
                        if (HospitalStay.iloc[b+1,0] - HospitalStay.iloc[b,0] <= 0):
                            # All durations should be positive non-zero values, since transitions should not be instantanous
                            print("NHS Number ="+str(i)+"Negative or zero length duration encountered, please check rounding in data")                          
                        if (qi==(jj-1)):
                            newrow = np.append(row,[HospitalStay.iloc[b+1,0] -
                                                    HospitalStay.iloc[b,0],ii,jj,1,qval]) # Observed transition
                        else:
                            newrow =  np.append(row,[HospitalStay.iloc[b+1,0] -
                                                     HospitalStay.iloc[b,0],ii,qi+1,0,qval]) # Possible transitions (not observed)
                        duplicated_df_vals.append(newrow)
            else:
                for qi, qval in enumerate(Qbase[ii-1]):
                    if (qval>0):
                        newrow =  np.append(row,[timepull,ii,qi+1,0,qval]) # Possible transitions (censored)
                        duplicated_df_vals.append(newrow)                    

        HospitalStayForFS = pd.DataFrame(duplicated_df_vals)
        NHS_nos = pd.DataFrame(np.repeat(i, len(HospitalStayForFS)))
        df_individual = pd.concat([NHS_nos, HospitalStayForFS], axis = 1)
        new_df = new_df.append(df_individual)
    except:
        print('Caught append error at step ' + str(l) + ' NHS Number =' +str(i))
        continue
new_df.columns = ["NHSNumber", "Time", "Age","ICU_cov", "State", "TimeInState", "From", "To", "Observed", "TransitionNumber"]

In [ ]:
#### Save length of stay data frame
new_df.to_csv('Current_COVID_'+file_date+'.csv')

In [ ]:
#### Define time series for census
num_trans = np.max(Qbase)
num_states = len(Qbase)
dtstart = datetime.fromisoformat('2020-03-01 00:00:01')
Tmax = int(np.floor((dtpull - dtstart).total_seconds()/(60*60*24))) 
newcensusb = np.zeros((Tmax, num_states + 1))
def days(x):
    # Returns days since dtstart
    return int((np.floor((x- dtstart).total_seconds()/(60*60*24))) )

In [ ]:
#### Create a 'census' of the hospital
na = 4 # number of age groups

output = np.zeros((Tmax+1,na*(num_states + 1)))

for j_index in tqdm(range(0,na)):
    newcensus = np.zeros((Tmax+1, num_states + 1))

    ids_covariates = new_df.loc[new_df['Age'] == j_index]
    ids_covariates = ids_covariates['NHSNumber'].unique()
    for i in tqdm(ids_covariates):
        NHS_df = df.loc[df['NHSNumber'] == i]
        Hosp_adt = NHS_df.HospitalAdmissionTime.values[0]
        Hosp_dt = NHS_df.HospitalDischargeTime.values[-1]
        ICU_admit = NHS_df.StartTimeCriticalCare.values
        ICU_discharge = NHS_df.DischargeTimeCriticalCare.values[-1]
        NHS_discharge = NHS_df.HospitalDischargeTime.values[-1]
        NHS_death = NHS_df.DateOfDeath.values[-1]

        # Logical indicator for death in hospital ward 

        died_in_hosp = False
        if (not pd.isna(NHS_death)):
            if (NHS_death <= NHS_discharge) :
                died_in_hosp = True

        ### Gives one more than observed in ward at end
        Hosp_adt_day = days(Hosp_adt)
        newcensus[Hosp_adt_day, 5] +=1

        if (not pd.isna(NHS_discharge)):
            Hosp_discharge_day = days(NHS_discharge)


        if np.any(~pd.isna(ICU_admit)): # Went to ICU
            ICU_starts = ICU_admit[~pd.isna(ICU_admit)]
            ICU_start = days(ICU_starts[0])
            newcensus[Hosp_adt_day:ICU_start, 0]+=1
        elif ((not died_in_hosp) and (not pd.isna(NHS_discharge))): #Discharged not died
            Hosp_dt_day = days(Hosp_dt)
            newcensus[Hosp_adt_day:Hosp_dt_day, 0]+=1
        elif(died_in_hosp): #Died
            Hosp_death_day = days(NHS_death)
            newcensus[Hosp_adt_day:Hosp_death_day, 0]+=1
        else: #Nothing Happened
            newcensus[Hosp_adt_day:, 0]+=1

        CriticalCare = NHS_df[['StartTimeCriticalCare','DischargeTimeCriticalCare']].values
        ccentries = CriticalCare[:,0]
        ccleaves = CriticalCare[:,1]
        jj = ~pd.isna(ccentries)
        ccentries = ccentries[jj]
        ccleaves = ccleaves[jj]
        for k in range(0, len(ccentries)):
            ICU_adt_day = days(ccentries[k])
            if (not pd.isna(ccleaves[k])):
                ICU_dt_day = days(ccleaves[k])
                newcensus[ICU_adt_day:ICU_dt_day, 1] +=1
            else:
                newcensus[ICU_adt_day:, 1] +=1

            if (k>0): # TO ADD TO STEPDOWN CENSUS
                ICU_return_day = days(ccentries[k])
                ICU_last_day = days(ccleaves[k-1])
                newcensus[ICU_last_day:ICU_return_day, 2] +=1

        if(not pd.isna(ICU_discharge)): # If not died in ICU, final stepdown
            ICU_dt_day = days(ICU_discharge)
            if (not pd.isna(NHS_discharge)):
                Hosp_dt_day = days(NHS_discharge)
                newcensus[ICU_dt_day:Hosp_dt_day, 2] +=1

            else:
                newcensus[ICU_dt_day:, 2] +=1

        if (not pd.isna(NHS_discharge)):
            if (died_in_hosp):
                death_day = days(NHS_discharge)
                newcensus[death_day:, 4] +=1
            else:
                discharge_day = days(NHS_discharge)
                newcensus[discharge_day:, 3] +=1
    output[:,j_index*(num_states+1):((j_index+1)*(num_states+1))] = newcensus

In [ ]:
#### Save census
pd.DataFrame((np.asarray(output))).to_csv('census_covariates_'+file_date+'.csv', index=False)

In [ ]:
#### Generate input file for Fit_Admissions_Growth.R

growth_len = 28 # number of days over which to estimate the growth rate trend. Try between 28 and 100.

R_df = np.zeros((growth_len,na+2))
R_df[:,0] = range(1,growth_len+1)
for index in range(0,na):
    R_df[:,index+1] = output[(Tmax-growth_len+1):(Tmax+1),((index+1)*6-1)]
R_df[:,na+1] = np.sum(R_df[:,1:na+1],axis=1)
R_df = pd.DataFrame((np.asarray(R_df)))
R_df.columns = ["dates", "counts_1", "counts_2","counts_3", "counts_4", "counts_5"]
R_df.to_csv('admissions_'+file_date+'.csv', index=False)